In [1]:
import numpy as np
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Embedding

2025-07-07 16:37:55.601465: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-07 16:37:55.624890: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751906275.650008    3028 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751906275.655087    3028 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751906275.677372    3028 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

Trying out with small made out data

In [2]:
reviews = ['nice food',
        'amazing restaurant',
        'too good',
        'just loved it!',
        'will go again',
        'horrible food',
        'never go there',
        'poor service',
        'poor quality',
        'needs improvement']

sentiment = np.array([1,1,1,1,1,0,0,0,0,0])

In [3]:
one_hot("amazing restaurant",30)

[26, 22]

keras one hot gave amazing as 7 and restaurant 12, randomly among 1-30

In [4]:
vocab_size = 30  # gives random numbering to words from 1-30
encoded_reviews = [one_hot(d, vocab_size) for d in reviews]  # simple list comprehension and performing one_hot on each elements
encoded_reviews

[[19, 10],
 [26, 22],
 [22, 6],
 [25, 23, 11],
 [25, 18, 25],
 [13, 10],
 [24, 18, 20],
 [13, 18],
 [13, 15],
 [28, 7]]

My given reviews are now encoded


Taking the max sentence size and perform padding so that neuron has static arhcitecture

In [5]:
max_length = 3
padded_reviews = pad_sequences(encoded_reviews, maxlen=max_length, padding='post')
print(padded_reviews)

[[19 10  0]
 [26 22  0]
 [22  6  0]
 [25 23 11]
 [25 18 25]
 [13 10  0]
 [24 18 20]
 [13 18  0]
 [13 15  0]
 [28  7  0]]


Now all of them are of same size

In [6]:
embeded_vector_size = 5  # Size of each word vector

model = Sequential()
model.add(Embedding(vocab_size, embeded_vector_size, name="embedding"))  # Turn words into vectors
model.add(Flatten())  # Flatten the 2D output into 1D
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

Making the variables X and y just to make it common practice after this

In [7]:
X = padded_reviews
y = sentiment

In [9]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.build(input_shape=(None, max_length))
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 3, 5)           │           150 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 15)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            16 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 166 (664.00 B)

 Trainable params: 166 (664.00 B)

 Non-trainable params: 0 (0.00 B)

In [10]:
model.fit(X, y, epochs=50, verbose=0)

I0000 00:00:1751906400.033217    3099 service.cc:152] XLA service 0x75777c005260 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1751906400.033282    3099 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce RTX 4050 Laptop GPU, Compute Capability 8.9
2025-07-07 16:40:00.090824: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1751906400.337297    3099 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1751906400.975811    3099 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [11]:
loss, accuracy = model.evaluate(X, y)
accuracy

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 715ms/step - accuracy: 1.0000 - loss: 0.5971


1.0

In [17]:
wts = model.get_layer("embedding").get_weights()[0]
wts

array([[ 0.01860758,  0.00530918, -0.00361782, -0.0180904 ,  0.02473282],
       [ 0.0339899 , -0.04253189, -0.00745092,  0.04166878,  0.04511345],
       [-0.00175641, -0.01460741, -0.01704478, -0.02380602, -0.02804444],
       [-0.00868622, -0.03307068, -0.02552002,  0.01995033, -0.01580159],
       [-0.02875379, -0.00807948, -0.04918018, -0.02344762,  0.04784309],
       [-0.02765667, -0.00527657, -0.02176096, -0.00952389,  0.0482204 ],
       [-0.0321962 , -0.05258087, -0.09716095, -0.04492914, -0.08304841],
       [ 0.02351359,  0.06117171,  0.08316506,  0.02764978,  0.05843484],
       [ 0.04997906, -0.00759761, -0.04056442, -0.00056816, -0.01400916],
       [-0.04558691, -0.00080311,  0.02859594,  0.00931307,  0.00502491],
       [ 0.04069296,  0.03336495, -0.00448856,  0.04334339,  0.00882232],
       [ 0.01140852,  0.0389963 ,  0.04318121,  0.07038363,  0.05525807],
       [ 0.04988662,  0.01081318, -0.01914179, -0.04985606, -0.01079569],
       [-0.13388366,  0.03593134, -0.1

In [16]:
len(wts)

30

Now checking the word embedding for few words (nice,amazing, poor)

In [18]:
wts[19]

array([ 0.07949639, -0.01512811,  0.09900574,  0.05377626,  0.08128713],
      dtype=float32)

In [21]:
wts[26]

array([ 0.07358743, -0.03702399,  0.09400085,  0.01665702,  0.01063377],
      dtype=float32)

Although nice and amazing are similar, the embedded vectors seems different here but in large datasets this will make sense

In [22]:
wts[13]

array([-0.13388366,  0.03593134, -0.10592492, -0.1340065 , -0.03768404],
      dtype=float32)